In [1]:
import pandas as pd
from pathlib import Path
import re
import traceback
from typing import Optional, List

# ==============================================================================
# 1. 配置层 (Configuration Layer)
# ==============================================================================
class Config:
    """集中管理所有文件路径、常量和筛选条件"""
    # --- 【已更新】输入和输出路径 ---
    INPUT_DIR = Path(r'E:\A智网\业扩分析\数据准备\充电桩用电_分行业售电') # <--- 已更新为您的新路径
    OUTPUT_FILE = Path(r'E:\A智网\业扩分析\26年1月分析\充电电量数据.xlsx') # <--- 已更新为您的新路径

    # --- 文件匹配模式 ---
    FILENAME_PATTERN = r'销售表5-3_全行业销售电量情况-(\d{4})\.xlsx?'

    # --- Excel读取参数 ---
    ROWS_TO_SKIP = 7
    SHEET_NAME = 0
    
    # --- 数据定位配置 ---
    INDEX_COL_SEQ = 0
    INDEX_COL_NAME = 1
    INDEX_COL_SALES = 6

    # --- 【请修改】要筛选的行业序号 ---
    TARGET_INDUSTRY_INDEXES = [6, 7, 8, 9, 10, 11, 12]

    # --- 单位转换 ---
    SALES_CONVERSION_FACTOR = 10000

# ==============================================================================
# 2. 辅助函数与数据处理层 (Helper & Processing Layer)
# ==============================================================================

def robust_to_numeric(series: pd.Series) -> pd.Series:
    """
    一个更健壮的函数，用于将可能包含逗号、空格等字符的文本列转换为数值类型。
    """
    s = series.astype(str)
    s = s.str.replace(',', '', regex=False)
    s = s.str.strip()
    return pd.to_numeric(s, errors='coerce')

def process_single_file(file_path: Path, month_str: str) -> Optional[pd.DataFrame]:
    """从单个文件中提取指定行业的数据"""
    print(f"  -> 正在处理文件: {file_path.name} (月份: {month_str})")
    try:
        df = pd.read_excel(
            file_path, 
            sheet_name=Config.SHEET_NAME, 
            skiprows=Config.ROWS_TO_SKIP, 
            header=None,
            dtype=str
        )

        df[Config.INDEX_COL_SEQ] = robust_to_numeric(df[Config.INDEX_COL_SEQ])
        df.dropna(subset=[Config.INDEX_COL_SEQ], inplace=True)
        
        df_filtered = df[df[Config.INDEX_COL_SEQ].isin(Config.TARGET_INDUSTRY_INDEXES)].copy()

        if df_filtered.empty:
            print(f"    [警告] 在文件中未找到任何指定的行业序号。")
            return None

        result = pd.DataFrame({
            '序号': df_filtered[Config.INDEX_COL_SEQ].astype(int),
            '行业': df_filtered[Config.INDEX_COL_NAME].astype(str).str.strip(),
            '月份': month_str,
            '售电量': robust_to_numeric(df_filtered[Config.INDEX_COL_SALES])
        })
        
        return result

    except Exception as e:
        print(f"    [错误] 处理文件 {file_path.name} 时发生未知错误。")
        print(traceback.format_exc())
        return None

# ==============================================================================
# 3. 主流程 (Main Workflow)
# ==============================================================================
def main():
    """主执行函数"""
    print("--- 开始执行销售电量数据提取任务 ---")

    if not Config.INPUT_DIR.exists():
        print(f"[致命错误] 输入文件夹不存在: {Config.INPUT_DIR}")
        return

    all_data = []
    file_paths = sorted(Config.INPUT_DIR.glob('*.xls*'))

    for file_path in file_paths:
        match = re.search(Config.FILENAME_PATTERN, file_path.name)
        if match:
            month_str = match.group(1)
            monthly_data = process_single_file(file_path, month_str)
            if monthly_data is not None:
                all_data.append(monthly_data)
    
    if not all_data:
        print("\n[处理中断] 未能从任何文件中成功提取数据。")
        return

    print("\n所有文件处理完毕，正在合并数据...")
    long_df = pd.concat(all_data, ignore_index=True)

    long_df['售电量'] = long_df['售电量'] / Config.SALES_CONVERSION_FACTOR
    long_df.fillna(0, inplace=True)
    print("数据合并与单位转换完成。")

    print("正在将数据重塑为最终报表格式...")
    final_report = long_df.pivot_table(
        index=['序号', '行业'],
        columns='月份',
        values='售电量',
        aggfunc='sum'
    )
    
    final_report.columns.name = None
    final_report = final_report.reset_index()
    
    print(f"正在将结果保存到: {Config.OUTPUT_FILE}")
    try:
        Config.OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)
        final_report.to_excel(Config.OUTPUT_FILE, index=False, engine='openpyxl')
        print(f"\n--- 全部任务成功完成！结果已保存到: {Config.OUTPUT_FILE} ---")
    except Exception as e:
        print(f"\n[致命错误] 保存Excel文件失败: {e}")
        print(traceback.format_exc())

if __name__ == '__main__':
    main()

--- 开始执行销售电量数据提取任务 ---
  -> 正在处理文件: 销售表5-3_全行业销售电量情况-2401.xlsx (月份: 2401)


d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


  -> 正在处理文件: 销售表5-3_全行业销售电量情况-2402.xlsx (月份: 2402)
  -> 正在处理文件: 销售表5-3_全行业销售电量情况-2403.xlsx (月份: 2403)
  -> 正在处理文件: 销售表5-3_全行业销售电量情况-2404.xlsx (月份: 2404)
  -> 正在处理文件: 销售表5-3_全行业销售电量情况-2405.xlsx (月份: 2405)


d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


  -> 正在处理文件: 销售表5-3_全行业销售电量情况-2406.xlsx (月份: 2406)
  -> 正在处理文件: 销售表5-3_全行业销售电量情况-2407.xlsx (月份: 2407)
  -> 正在处理文件: 销售表5-3_全行业销售电量情况-2408.xlsx (月份: 2408)
  -> 正在处理文件: 销售表5-3_全行业销售电量情况-2409.xlsx (月份: 2409)


...

d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


  -> 正在处理文件: 销售表5-3_全行业销售电量情况-2506.xlsx (月份: 2506)
  -> 正在处理文件: 销售表5-3_全行业销售电量情况-2507.xlsx (月份: 2507)
  -> 正在处理文件: 销售表5-3_全行业销售电量情况-2508.xlsx (月份: 2508)


d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


  -> 正在处理文件: 销售表5-3_全行业销售电量情况-2509.xlsx (月份: 2509)
  -> 正在处理文件: 销售表5-3_全行业销售电量情况-2510.xlsx (月份: 2510)
  -> 正在处理文件: 销售表5-3_全行业销售电量情况-2511.xlsx (月份: 2511)
  -> 正在处理文件: 销售表5-3_全行业销售电量情况-2512.xlsx (月份: 2512)

所有文件处理完毕，正在合并数据...
数据合并与单位转换完成。
正在将数据重塑为最终报表格式...
正在将结果保存到: E:\A智网\业扩分析\26年1月分析\充电电量数据.xlsx

--- 全部任务成功完成！结果已保存到: E:\A智网\业扩分析\26年1月分析\充电电量数据.xlsx ---


d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
import pandas as pd
from pathlib import Path
import numpy as np
import traceback

# ==============================================================================
# 1. 配置层 (Configuration Layer)
# ==============================================================================
class Config:
    """集中管理所有文件路径、Sheet名和列名"""
    # --- 【请修改】文件路径和目标月份 ---
    # 注意：输入和输出是同一个文件
    FILE_PATH = Path(r'E:\A智网\业扩分析\26年1月分析\充电电量数据.xlsx')
    
    # !! 关键 !!: 每次运行时，只需要修改这里的目标月份
    TARGET_MONTH = '202512'

    # --- Sheet名称 ---
    INPUT_SHEET_NAME = '电量'
    OUTPUT_SHEET_NAME = '同比结果'

    # --- 关键列名 ---
    SEQ_COL = '序号'
    CATEGORY_COL = '行业'
    
    # --- 输出列名 ---
    OUTPUT_COL_MONTHLY_YOY = '当月电量同比'
    OUTPUT_COL_3MONTH_YOY = '近三月电量同比'

# ==============================================================================
# 2. 辅助函数层 (Helper Functions)
# ==============================================================================
def calculate_percentage_change(current, previous):
    """健壮的百分比变化计算函数，处理除以零或数据缺失的情况"""
    # 如果任一值为NaN或None，则无法计算
    if pd.isna(current) or pd.isna(previous):
        return np.nan # 返回NaN，以便后续可以格式化为'N/A'
    
    # 如果去年同期值为0
    if previous == 0:
        # 如果今年值也为0，则无变化
        if current == 0:
            return 0.0
        # 如果今年值不为0，则为无穷大增长，通常表示为'N/A'或特殊标记
        else:
            return np.inf # 返回无穷大
            
    change = (current - previous) / abs(previous)
    return change

# =' '='============================================================================
# 3. 主流程 (Main Workflow)
# ==============================================================================
def main():
    """主执行函数"""
    print(f"--- 开始为月份 {Config.TARGET_MONTH} 计算电量同比 ---")
    print(f"--- 处理文件: {Config.FILE_PATH} ---")

    if not Config.FILE_PATH.exists():
        print(f"[致命错误] 输入文件不存在: {Config.FILE_PATH}")
        return

    try:
        # 1. 加载数据
        print(f"正在加载Sheet '{Config.INPUT_SHEET_NAME}'...")
        df = pd.read_excel(Config.FILE_PATH, sheet_name=Config.INPUT_SHEET_NAME)
        
        # --- 【核心修正】将所有列名强制转换为字符串类型 ---
        df.columns = df.columns.map(str)
        print("  -> 已将所有列名转换为字符串格式以确保匹配。")
        
        # 2. 动态计算所有相关月份的列名
        print("正在计算相关月份...")
        current_year = int(Config.TARGET_MONTH[:4])
        current_month_num = int(Config.TARGET_MONTH[4:])
        
        target_month_col = Config.TARGET_MONTH
        last_year_month_col = f"{current_year - 1}{current_month_num:02d}"
        
        last_3_months_current_year = []
        for i in range(3):
            month_dt = pd.to_datetime(target_month_col, format='%Y%m') - pd.DateOffset(months=i)
            last_3_months_current_year.append(month_dt.strftime('%Y%m'))
        
        last_3_months_previous_year = [str(int(m[:4]) - 1) + m[4:] for m in last_3_months_current_year]

        print(f"  - 当月: {target_month_col}, 上年同期: {last_year_month_col}")
        print(f"  - 今年近三月: {last_3_months_current_year}")
        print(f"  - 去年同期近三月: {last_3_months_previous_year}")

        # 3. 检查所有需要的列是否存在
        all_required_cols = [target_month_col, last_year_month_col] + last_3_months_current_year + last_3_months_previous_year
        missing_cols = [col for col in all_required_cols if col not in df.columns]
        if missing_cols:
            print(f"\n[严重警告] 源数据Sheet '{Config.INPUT_SHEET_NAME}' 中缺少以下必需的月份列: {', '.join(sorted(list(set(missing_cols))))}")
            print("程序将继续运行，但缺少数据的计算结果将为空。")
            for col in missing_cols:
                df[col] = np.nan

        # 4. 创建结果DataFrame
        result_df = df[[Config.SEQ_COL, Config.CATEGORY_COL]].copy()

        # 5. 计算“当月电量同比”
        print("\n正在计算“当月电量同比”...")
        current_month_load = pd.to_numeric(df[target_month_col], errors='coerce')
        last_year_month_load = pd.to_numeric(df[last_year_month_col], errors='coerce')
        result_df[Config.OUTPUT_COL_MONTHLY_YOY] = [
            calculate_percentage_change(cur, prev) 
            for cur, prev in zip(current_month_load, last_year_month_load)
        ]
        
        # 6. 计算“近三月电量同比”
        print("正在计算“近三月电量同比”...")
        current_3m_sum = df[last_3_months_current_year].apply(pd.to_numeric, errors='coerce').sum(axis=1, skipna=False)
        previous_3m_sum = df[last_3_months_previous_year].apply(pd.to_numeric, errors='coerce').sum(axis=1, skipna=False)
        
        result_df[Config.OUTPUT_COL_3MONTH_YOY] = [
            calculate_percentage_change(cur, prev)
            for cur, prev in zip(current_3m_sum, previous_3m_sum)
        ]
        
        # 7. 格式化百分比输出
        print("正在格式化输出结果...")
        for col in [Config.OUTPUT_COL_MONTHLY_YOY, Config.OUTPUT_COL_3MONTH_YOY]:
            result_df[col] = result_df[col].apply(lambda x: f"{x:.2%}" if pd.notna(x) and np.isfinite(x) else 'N/A')

        # 8. 将结果写入到原始Excel文件的新Sheet中
        print(f"正在将结果写入到文件 '{Config.FILE_PATH.name}' 的新Sheet '{Config.OUTPUT_SHEET_NAME}'...")
        with pd.ExcelWriter(Config.FILE_PATH, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            result_df.to_excel(writer, sheet_name=Config.OUTPUT_SHEET_NAME, index=False)
        
        print(f"\n--- 全部任务成功完成！ ---")
        print(f"请打开文件 '{Config.FILE_PATH}' 并检查名为 '{Config.OUTPUT_SHEET_NAME}' 的工作表。")

    except FileNotFoundError:
        print(f"[致命错误] 无法找到输入文件: {Config.FILE_PATH}")
    except ValueError as e:
        if f"Worksheet {Config.INPUT_SHEET_NAME} does not exist" in str(e):
             print(f"[致命错误] 在Excel文件中找不到名为 '{Config.INPUT_SHEET_NAME}' 的工作表。")
        else:
            print(f"[致命错误] 执行过程中发生值错误: {e}")
            print(traceback.format_exc())
    except Exception as e:
        print(f"[致命错误] 执行过程中发生未知错误。")
        print(traceback.format_exc())

if __name__ == '__main__':
    main()

--- 开始为月份 202512 计算电量同比 ---
--- 处理文件: E:\A智网\业扩分析\26年1月分析\充电电量数据.xlsx ---
正在加载Sheet '电量'...
  -> 已将所有列名转换为字符串格式以确保匹配。
正在计算相关月份...
  - 当月: 202512, 上年同期: 202412
  - 今年近三月: ['202512', '202511', '202510']
  - 去年同期近三月: ['202412', '202411', '202410']

正在计算“当月电量同比”...
正在计算“近三月电量同比”...
正在格式化输出结果...
正在将结果写入到文件 '充电电量数据.xlsx' 的新Sheet '同比结果'...

--- 全部任务成功完成！ ---
请打开文件 'E:\A智网\业扩分析\26年1月分析\充电电量数据.xlsx' 并检查名为 '同比结果' 的工作表。
